#Customer Churn Anaylsis

##PDSML day 2 - importing libraries and datasets

In [1]:
a=103
b=199
c=a+b
print(c)

302


###importing basic libraries

In [2]:
import math
a=math.sqrt(245)

print(math.ceil(a))


16


###importing essential libs for data analysis


In [3]:
import pandas as pd           #descriptive analysis immporting the dataset using it
import numpy as np            #mathematical functions
import matplotlib.pyplot as plt #for ploting our visuals
from sklearn.model_selection import train_test_split #this will help to slpit the data into training set and testing set
from imblearn.over_sampling import SMOTE   #synthetic minority over sampling technique #imbalance learning
import shap                                # this will help to find the factors responsible for churn





C:\Users\Chandan Hembrom\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


###importing dataset

In [4]:
dataset=pd.read_csv("Telco-Customer-Churn.csv")



In [5]:
print(f"the shape of the dataset: {dataset.shape}")

the shape of the dataset: (7043, 21)


###checking value_counts()


In [6]:
dataset['InternetService'].value_counts(normalize=True)*100

InternetService
Fiber optic    43.958540
DSL            34.374556
No             21.666903
Name: proportion, dtype: float64

##Day3 Data cleaning

###Detecting error erroneous numeric columns by calculating the mean

In [7]:
print(f"mean of tenure {dataset['tenure'].mean()}")


mean of tenure 32.37114865824223


In [8]:
print(dataset['MonthlyCharges'].mean())

64.76169246059918


###printting all the list of the blank entry in totalCharges

In [9]:
j=0
for i in range (7042):
  if dataset['TotalCharges'].iloc[i]==' ':
    print(f"the blank is in {i+2}")
    j+=1

print(f"the count is : {j}")


the blank is in 490
the blank is in 755
the blank is in 938
the blank is in 1084
the blank is in 1342
the blank is in 3333
the blank is in 3828
the blank is in 4382
the blank is in 5220
the blank is in 6672
the blank is in 6756
the count is : 11


###converting the blanks into nan so it can be ignored will calculating the mean

In [10]:
dataset['TotalCharges']=pd.to_numeric(dataset['TotalCharges'],errors='coerce')
print(f"the mean: {dataset['TotalCharges'].mean()}")
dataset.shape

the mean: 2283.3004408418656


(7043, 21)

###it is easy to just delete the row with the nan value. so now we drop the nan containing row and check the shape of the dataset afterwards

In [11]:
dataset=dataset.dropna(subset=['TotalCharges']).copy()
dataset.shape

(7032, 21)

##Day4 PDSML Day-4 👉 Data Preprocessing (Part-2)


###Removing unneccessary details


In [12]:
dataset.head() #no showing all the columns


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [13]:
list(dataset.columns) # now we can see all the columns

['customerID',
 'gender',
 'SeniorCitizen',
 'Partner',
 'Dependents',
 'tenure',
 'PhoneService',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

### Drop columns



In [14]:
X=dataset.drop(columns=["customerID","Churn"])

X.shape

(7032, 19)

### Dependent variable Declaration


In [15]:
print(dataset['Churn'].head())

0     No
1     No
2    Yes
3     No
4    Yes
Name: Churn, dtype: object


In [16]:
y=dataset['Churn']


### mapping the yes no to 1 0

In [17]:
y=y.map({'No':0,'Yes':1}) #curly brackets means dictionary structure
print(y)


0       0
1       0
2       1
3       0
4       1
       ..
7038    0
7039    0
7040    0
7041    1
7042    0
Name: Churn, Length: 7032, dtype: int64


###Split the dataset into test set and traning set

In [18]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)



In [19]:
print(X_trainset.head())

      gender  SeniorCitizen Partner Dependents  tenure PhoneService  \
1413    Male              0     Yes        Yes      65          Yes   
7003    Male              0      No         No      26           No   
3355  Female              0     Yes         No      68          Yes   
4494    Male              0      No         No       3          Yes   
3541  Female              0     Yes         No      49           No   

         MultipleLines InternetService OnlineSecurity OnlineBackup  \
1413               Yes     Fiber optic            Yes          Yes   
7003  No phone service             DSL             No           No   
3355               Yes     Fiber optic             No          Yes   
4494                No     Fiber optic             No          Yes   
3541  No phone service             DSL            Yes           No   

     DeviceProtection TechSupport StreamingTV StreamingMovies        Contract  \
1413              Yes         Yes          No              No        Tw

In [20]:
y.value_counts(normalize=True)


Churn
0    0.734215
1    0.265785
Name: proportion, dtype: float64

In [21]:
y_trainset.value_counts(normalize=True)

Churn
0    0.734222
1    0.265778
Name: proportion, dtype: float64

##Day-5 👉 Data Preprocessing (Part-3)

###One hot encoding

In [22]:
#done it using chatGpt

encoded_data=pd.get_dummies(dataset,columns=['InternetService'],dtype=int)

print(encoded_data)

      customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines OnlineSecurity OnlineBackup  ...  \
0              No  No phone service             No          Yes  ...   
1             Y

In [23]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder






####Selecting numerical columns

In [24]:
numeric_cols=X.select_dtypes(include=np.number).columns.tolist()

print(numeric_cols)

['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']


####Selecting the Categorical columns


In [25]:
categorical_cols=X.select_dtypes(exclude=np.number).columns.tolist()
print(categorical_cols)

['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']


In [26]:
pre_processed=ColumnTransformer(transformers=[
    ("cat",OneHotEncoder(handle_unknown='ignore',sparse_output=False),categorical_cols),
    ("num","passthrough",numeric_cols)
])

In [27]:
X_preprocessed=pre_processed.fit_transform(X)
print(X_preprocessed)

[[1.0000e+00 0.0000e+00 0.0000e+00 ... 1.0000e+00 2.9850e+01 2.9850e+01]
 [0.0000e+00 1.0000e+00 1.0000e+00 ... 3.4000e+01 5.6950e+01 1.8895e+03]
 [0.0000e+00 1.0000e+00 1.0000e+00 ... 2.0000e+00 5.3850e+01 1.0815e+02]
 ...
 [1.0000e+00 0.0000e+00 0.0000e+00 ... 1.1000e+01 2.9600e+01 3.4645e+02]
 [0.0000e+00 1.0000e+00 0.0000e+00 ... 4.0000e+00 7.4400e+01 3.0660e+02]
 [0.0000e+00 1.0000e+00 1.0000e+00 ... 6.6000e+01 1.0565e+02 6.8445e+03]]


##Day6 Data preprocessing (part -4)

###Getting names of encoded categorical columns

In [28]:
cat_iv_names=pre_processed.named_transformers_['cat'].get_feature_names_out(categorical_cols)

In [29]:
print(numeric_cols)

['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges']


###Combining categorical and numerical clumns

In [30]:
all_iv_names=list(cat_iv_names)+numeric_cols #the Cat_iv_name is not list. so need to convert it to a list

print(all_iv_names)

['gender_Female', 'gender_Male', 'Partner_No', 'Partner_Yes', 'Dependents_No', 'Dependents_Yes', 'PhoneService_No', 'PhoneService_Yes', 'MultipleLines_No', 'MultipleLines_No phone service', 'MultipleLines_Yes', 'InternetService_DSL', 'InternetService_Fiber optic', 'InternetService_No', 'OnlineSecurity_No', 'OnlineSecurity_No internet service', 'OnlineSecurity_Yes', 'OnlineBackup_No', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No', 'DeviceProtection_No internet service', 'DeviceProtection_Yes', 'TechSupport_No', 'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No', 'StreamingTV_No internet service', 'StreamingTV_Yes', 'StreamingMovies_No', 'StreamingMovies_No internet service', 'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year', 'Contract_Two year', 'PaperlessBilling_No', 'PaperlessBilling_Yes', 'PaymentMethod_Bank transfer (automatic)', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Electronic check', 'Payment

###Converting the transformed output to a DataFrame

In [31]:
X_tranformed=pre_processed.fit_transform(X)


In [32]:
print(X_tranformed)

[[1.0000e+00 0.0000e+00 0.0000e+00 ... 1.0000e+00 2.9850e+01 2.9850e+01]
 [0.0000e+00 1.0000e+00 1.0000e+00 ... 3.4000e+01 5.6950e+01 1.8895e+03]
 [0.0000e+00 1.0000e+00 1.0000e+00 ... 2.0000e+00 5.3850e+01 1.0815e+02]
 ...
 [1.0000e+00 0.0000e+00 0.0000e+00 ... 1.1000e+01 2.9600e+01 3.4645e+02]
 [0.0000e+00 1.0000e+00 0.0000e+00 ... 4.0000e+00 7.4400e+01 3.0660e+02]
 [0.0000e+00 1.0000e+00 1.0000e+00 ... 6.6000e+01 1.0565e+02 6.8445e+03]]


In [34]:
tr_df=pd.DataFrame(X_tranformed,columns=all_iv_names)
pd.set_option('display.max_columns',None)

print(tr_df)



      gender_Female  gender_Male  Partner_No  Partner_Yes  Dependents_No  \
0               1.0          0.0         0.0          1.0            1.0   
1               0.0          1.0         1.0          0.0            1.0   
2               0.0          1.0         1.0          0.0            1.0   
3               0.0          1.0         1.0          0.0            1.0   
4               1.0          0.0         1.0          0.0            1.0   
...             ...          ...         ...          ...            ...   
7027            0.0          1.0         0.0          1.0            0.0   
7028            1.0          0.0         0.0          1.0            0.0   
7029            1.0          0.0         0.0          1.0            0.0   
7030            0.0          1.0         0.0          1.0            1.0   
7031            0.0          1.0         1.0          0.0            1.0   

      Dependents_Yes  PhoneService_No  PhoneService_Yes  MultipleLines_No  \
0         